# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install -q "datasets==2.15.0" transformers peft pandas datasets numpy scikit-learn


In [30]:
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_dataset
import pandas as pd
import numpy as np

# Load the train and test splits of the rotten_tomatoes dataset, Attributed to the Udacity course code and Huggingface code snippets.
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("rotten_tomatoes", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Pre-process dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)

# Load and set up the base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

# Freeze all the parameters of the base model
for param in base_model.base_model.parameters():
    param.requires_grad = False


Map: 100%|██████████| 500/500 [00:00<00:00, 11854.21 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
print(ds)

{'train': Dataset({
    features: ['text', 'label'],
    num_rows: 500
}), 'test': Dataset({
    features: ['text', 'label'],
    num_rows: 500
})}


In [34]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}
    
    
# Training the base model, Attributed to the Udacity course code.
trainer_base = Trainer(
    model=base_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis_base",
        learning_rate=2e-3,
        per_device_train_batch_size=6,
        per_device_eval_batch_size=6,
        num_train_epochs=2,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Training the base model
trainer_base.train()

# Evaluate the base model
base_model_evaluation = trainer_base.evaluate()

/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/b4/7fl6nhb926s15j28p0z4y2_40000gn/T/ipykernel_44965/3459446202.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_base = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.710961,0.592000
2,No log,0.526036,0.750000


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [35]:
# Performing Parameter-Efficient Fine-Tuning (PEFT)

# Unfreeze all the model parameters.
for param in base_model.parameters():
    param.requires_grad = True

# Training the PEFT model
trainer_peft = Trainer(
    model=base_model,  # Use the base model with unfrozen parameters
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis_peft", # Output directory for saving the PEFT model
        learning_rate=2e-5,
        per_device_train_batch_size=12,
        per_device_eval_batch_size=12,
        num_train_epochs=4,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer_peft.train()

/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/b4/7fl6nhb926s15j28p0z4y2_40000gn/T/ipykernel_44965/210267730.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_peft = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693612,0.696000
2,No log,0.686024,0.774000
3,No log,0.871264,0.780000
4,No log,1.043395,0.782000


TrainOutput(global_step=168, training_loss=0.16967939195178805, metrics={'train_runtime': 95.3607, 'train_samples_per_second': 20.973, 'train_steps_per_second': 1.762, 'total_flos': 264934797312000.0, 'train_loss': 0.16967939195178805, 'epoch': 4.0})

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [36]:
# Evaluate the PEFT model
peft_model_evaluation = trainer_peft.evaluate()

# Compare the results of the base model and PEFT model
print("Base Model Evaluation:")
print(base_model_evaluation)

print("\nPEFT Model Evaluation:")
print(peft_model_evaluation)

Base Model Evaluation:
{'eval_loss': 0.5260359048843384, 'eval_accuracy': 0.75, 'eval_runtime': 5.9025, 'eval_samples_per_second': 84.71, 'eval_steps_per_second': 14.231, 'epoch': 2.0}

PEFT Model Evaluation:
{'eval_loss': 0.6860244870185852, 'eval_accuracy': 0.774, 'eval_runtime': 5.6357, 'eval_samples_per_second': 88.72, 'eval_steps_per_second': 7.453, 'epoch': 4.0}
